In [1]:
import env
import pandas as pd
import numpy as np
import wrangle_zillow
from sklearn.impute import KNNImputer

In [2]:
# url = env.get_db_url('zillow')
# query = '''
# SELECT * FROM predictions_2017
# LEFT JOIN properties_2017 USING (parcelid)
# LEFT JOIN airconditioningtype USING (airconditioningtypeid)
# LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
# LEFT JOIN buildingclasstype USING (buildingclasstypeid)
# LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
# LEFT JOIN propertylandusetype USING (propertylandusetypeid)
# LEFT JOIN storytype USING (storytypeid)
# LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
# WHERE (latitude IS NOT NULL AND 
# 		longitude IS NOT NULL)'''
# df_orig = pd.read_sql(query, url)

In [3]:
# df = df_orig.copy()

In [4]:
# new_dates = \
# df.groupby(by='parcelid').transactiondate.max().reset_index()

In [5]:
# df.drop(columns=['parcelid','transactiondate'], inplace=True)

In [6]:
# df = new_dates.join(df, how='left')

In [7]:
df = wrangle_zillow.get_zillow_data()

In [8]:
df.shape

(77381, 67)

In [9]:
df.parcelid.is_unique

True

In [10]:
# df.drop(columns='id', inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 67 columns):
parcelid                        77381 non-null int64
transactiondate                 77381 non-null object
typeconstructiontypeid          223 non-null float64
storytypeid                     50 non-null float64
propertylandusetypeid           77381 non-null float64
heatingorsystemtypeid           49385 non-null float64
buildingclasstypeid             15 non-null float64
architecturalstyletypeid        207 non-null float64
airconditioningtypeid           24927 non-null float64
logerror                        77381 non-null float64
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingqualitytypeid           49617 non-null float64
calculatedbathnbr               76766 non-null float64
decktypeid                      614 non-null float64
finishedfloor1squarefeet     

In [12]:
# summary = pd.DataFrame(df.columns)
# summary['num_rows_missing'] = df.isna().sum().values
# summary['pct_rows_missing'] = df.isna().sum().values / len(df)
# summary.set_index(0, inplace=True)
# summary.index.name=''
# summary.head()

In [13]:
summary = wrangle_zillow.null_rows_info(df)
summary.pct_rows_missing.nlargest(5)


buildingclasstypeid     0.999806
buildingclassdesc       0.999806
finishedsquarefeet13    0.999457
storytypeid             0.999354
basementsqft            0.999354
Name: pct_rows_missing, dtype: float64

I likely want to just drop the columns that are missing at least 70% of their data.

In [14]:
# summary = pd.DataFrame(df.isna().sum(axis=1).values)
# summary.reset_index(inplace=True)
# summary.rename(columns={0:'num_cols_missing'}, inplace=True)
# df2 = summary.groupby('num_cols_missing')\
#                              .count().reset_index()
# df2['pct_cols_missing'] = df2.num_cols_missing / df.shape[1]
# df2.rename(columns={'index':'num_rows'}, inplace=True)
# df2

In [15]:
summary2 = wrangle_zillow.null_cols_info(df)
summary2

,num_cols_missing,num_rows,pct_cols_missing
0,23,2,0.343284
1,24,13,0.358209
2,25,24,0.373134
3,26,66,0.388060
4,27,315,0.402985
5,28,457,0.417910
6,29,5270,0.432836
7,30,3454,0.447761
8,31,9876,0.462687
9,32,12572,0.477612


It would be easy enough to drop rows that are missing more than 60% of their data, because that only accounts for a few hundred rows out of thousands.

In [16]:
single_unit = [261, 262, 263, 264, 268, 273, 276, 279]
df2 = df[df.propertylandusetypeid.isin(single_unit)]
df2.shape

(52387, 67)

In [17]:
df2.shape[0] - int(round(.7*len(df2),0))

15716

In [18]:
df2.isna().sum()[df2.isna().sum()>18000]

typeconstructiontypeid      52311
storytypeid                 52340
heatingorsystemtypeid       18576
buildingclasstypeid         52387
architecturalstyletypeid    52317
airconditioningtypeid       38783
basementsqft                52340
buildingqualitytypeid       18721
decktypeid                  51997
finishedfloor1squarefeet    48000
finishedsquarefeet13        52385
finishedsquarefeet15        52387
finishedsquarefeet50        48000
finishedsquarefeet6         52222
fireplacecnt                45144
garagecarcnt                34366
garagetotalsqft             34366
hashottuborspa              50872
poolcnt                     41312
poolsizesum                 51520
pooltypeid10                51943
pooltypeid2                 51316
pooltypeid7                 42399
propertyzoningdesc          18612
regionidneighborhood        33410
threequarterbathnbr         45663
unitcnt                     18613
yardbuildingsqft17          50450
yardbuildingsqft26          52322
numberofstorie

In [19]:
wrangle_zillow.handle_missing_values(df2)
df2.shape

/Users/camerontaylor/codeup-data-science/ds-methodologies-exercises/clustering/wrangle_zillow.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=1, thresh=threshold, inplace=True)
/Users/camerontaylor/codeup-data-science/ds-methodologies-exercises/clustering/wrangle_zillow.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0, thresh=threshold, inplace=True)


(52387, 28)

In [20]:
# def handle_missing_values(df, req_column = .7, req_row = .6):
#     threshold = int(round(req_column*len(df),0))
#     df.dropna(axis=1, thresh=threshold, inplace=True)
    
#     threshold = int(round(req_row*len(df.columns),0))
#     df.dropna(axis=0, thresh=threshold, inplace=True)
#     return df

In [21]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52387 entries, 0 to 77380
Data columns (total 28 columns):
parcelid                        52387 non-null int64
transactiondate                 52387 non-null object
propertylandusetypeid           52387 non-null float64
logerror                        52387 non-null float64
bathroomcnt                     52387 non-null float64
bedroomcnt                      52387 non-null float64
calculatedbathnbr               52237 non-null float64
calculatedfinishedsquarefeet    52292 non-null float64
finishedsquarefeet12            52125 non-null float64
fips                            52387 non-null float64
fullbathcnt                     52237 non-null float64
latitude                        52387 non-null float64
longitude                       52387 non-null float64
lotsizesquarefeet               52011 non-null float64
propertycountylandusecode       52387 non-null object
rawcensustractandblock          52387 non-null float64
regionidcity   

In [22]:
df2.isna().sum()[df2.isna().sum() > 0].index.tolist()

['calculatedbathnbr',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'fullbathcnt',
 'lotsizesquarefeet',
 'regionidcity',
 'regionidzip',
 'yearbuilt',
 'structuretaxvaluedollarcnt',
 'taxvaluedollarcnt',
 'landtaxvaluedollarcnt',
 'taxamount',
 'censustractandblock']

In [23]:
na_cols = df2[['parcelid','calculatedbathnbr', 
       'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'fullbathcnt', 'lotsizesquarefeet',
       'regionidcity', 'regionidzip', 'yearbuilt',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt',
       'landtaxvaluedollarcnt', 'taxamount', 'censustractandblock']]
na_cols = pd.DataFrame(KNNImputer().fit_transform(na_cols))

In [24]:
na_cols.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
8107,11180292.0,2.5,1853.0,1853.0,2.0,4000.0,27110.0,97118.0,1986.0,259370.0,518740.0,259370.0,6003.02,6.111008e+13
9128,11241249.0,2.0,1885.0,1885.0,2.0,8751.0,51239.0,97089.0,1994.0,360365.0,600607.0,240242.0,6476.68,6.111005e+13
6826,11119391.0,3.5,3667.0,3667.0,3.0,5261.0,15554.0,96993.0,1998.0,450000.0,1026000.0,576000.0,14477.98,6.059032e+13
276,10727338.0,1.0,882.0,882.0,1.0,10843.0,12447.0,96364.0,1952.0,57742.0,288725.0,230983.0,3602.79,6.037132e+13
4463,10967262.0,1.5,1124.0,1124.0,1.0,2254.0,42967.0,96203.0,1972.0,62865.0,350000.0,287135.0,4339.64,6.059088e+13


In [25]:
na_cols.columns = ['parcelid','calculatedbathnbr', 
       'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'fullbathcnt', 'lotsizesquarefeet',
       'regionidcity', 'regionidzip', 'yearbuilt',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt',
       'landtaxvaluedollarcnt', 'taxamount', 'censustractandblock']
int_cols = ['parcelid', 'calculatedfinishedsquarefeet',
        'finishedsquarefeet12','fullbathcnt','lotsizesquarefeet',
        'regionidcity','regionidzip', 'yearbuilt',
        'censustractandblock']
na_cols[int_cols] = na_cols[int_cols].astype(int)
na_cols.head()

,parcelid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fullbathcnt,lotsizesquarefeet,regionidcity,regionidzip,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,censustractandblock
0,10711855,3.5,3100,3100,3,4506,53571,96978,1998,485713.0,1023282.0,537569.0,11013.72,60590630072012
1,10711877,1.0,1465,1465,1,12647,13091,97099,1967,88000.0,464000.0,376000.0,5672.48,61110010023006
2,10711888,2.0,1243,1243,2,8432,21412,97078,1962,85289.0,564778.0,479489.0,6488.30,60590218022012
3,10711910,3.0,2376,2376,3,13038,396551,96330,1970,108918.0,145143.0,36225.0,1777.51,60373001001006
4,10711956,3.0,2962,2962,3,63000,47019,96293,1950,276684.0,773303.0,496619.0,9516.26,60374608001014


In [26]:
df3 = na_cols.merge(df2, on='parcelid', copy=False)
df3.shape

(52387, 41)

In [27]:
df3 = df3.drop(columns=df3.isna().sum()[df3.isna().sum()>0]\
               .index.tolist())
df3.isna().sum()

parcelid                          0
calculatedbathnbr_x               0
calculatedfinishedsquarefeet_x    0
finishedsquarefeet12_x            0
fullbathcnt_x                     0
lotsizesquarefeet_x               0
regionidcity_x                    0
regionidzip_x                     0
yearbuilt_x                       0
structuretaxvaluedollarcnt_x      0
taxvaluedollarcnt_x               0
landtaxvaluedollarcnt_x           0
taxamount_x                       0
censustractandblock_x             0
transactiondate                   0
propertylandusetypeid             0
logerror                          0
bathroomcnt                       0
bedroomcnt                        0
fips                              0
latitude                          0
longitude                         0
propertycountylandusecode         0
rawcensustractandblock            0
regionidcounty                    0
roomcnt                           0
assessmentyear                    0
propertylandusedesc         

In [28]:
# These are essentially duplicated columns
df3.drop(columns=['calculatedbathnbr_x', 'propertylandusetypeid',
                 'finishedsquarefeet12_x'], inplace=True)

In [29]:
df3.shape

(52387, 25)

In [34]:
df.regionidcity.value_counts()

12447.0     17934
46298.0      2589
5534.0       1913
54311.0      1871
52650.0      1670
            ...  
13311.0         5
118880.0        5
10815.0         4
21395.0         1
36078.0         1
Name: regionidcity, Length: 175, dtype: int64

In [40]:
df3.regionidcity_x.value_counts()
#DO NOT IMPUTE THIS

12447    11414
5534      1807
40227     1498
46298     1424
16764     1087
         ...  
19034        1
27166        1
22872        1
28797        1
12358        1
Name: regionidcity_x, Length: 826, dtype: int64

In [ ]:
plt.plot()